In [1]:
import numpy as np

import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

In [2]:
with open('results_num_19.npy', 'rb') as f:
    inputs = np.load(f)
    outputs = np.load(f)

In [3]:
inputs.shape, outputs.shape

((407, 2, 72, 72, 72), (407, 1, 72, 72, 72))

In [4]:
new_input = inputs[:, 0, 35, :, :]
new_input.shape

(407, 72, 72)

In [5]:
new_output = outputs[:, 0, 35, :, :]
new_output.shape

(407, 72, 72)

In [6]:
cats = pd.qcut(new_input.flatten(), q=30)

In [7]:
color_interval_values = []
color = 1
# i = 1
for interval in list(cats.categories):
    color_interval_values.append([interval.left, interval.right, color])
#     if i % 2 == 0:
    color += 1
#     i += 1

In [8]:
color_intervals_df = pd.DataFrame(color_interval_values, columns=['left', 'right', 'color'])
color_intervals_df

,left,right,color
0,-0.00100,0.00273,1
1,0.00273,0.00370,2
2,0.00370,0.00453,3
3,0.00453,0.00533,4
4,0.00533,0.00613,5
5,0.00613,0.00696,6
6,0.00696,0.00785,7
7,0.00785,0.00879,8
8,0.00879,0.00980,9
9,0.00980,0.01090,10


In [9]:
def estimate_color(x):
    for index, row in color_intervals_df.iterrows():
        if x > row.left and x <= row.right:
            return row.color
    if x <= color_intervals_df.iloc[0]['left']:
        return 1
    else:
        return 30

In [10]:
# location_value = []
# frame = 0
# index = 1

# for i in range(new_input[index].shape[0]):
#     for j in range(new_input[index].shape[1]):
#         location_value.append([i, j, new_input[index, i, j]])

# df = pd.DataFrame(data = location_value, columns=['x', 'y', 'value'])

# # df = df[df['value']>0.1]

# df['color'] = df['value'].apply(estimate_color)

# fig = px.scatter(df, x='x', y='y', color='color', range_color = [1, 30], color_continuous_scale='Viridis', width=800, height=800)

# fig.update_traces(marker=dict(size=12), selector=dict(mode='markers'))

# fig.show()

In [11]:
# location_value = []
# frame = 0
# index = 0

# for i in range(new_output[index].shape[0]):
#     for j in range(new_output[index].shape[1]):
#         location_value.append([i, j, new_output[index, i, j]])

# df = pd.DataFrame(data = location_value, columns=['x', 'y', 'value'])

# # df = df[df['value']>0.1]

# df['color'] = df['value'].apply(estimate_color)

# fig = px.scatter(df, x='x', y='y', color='color', range_color = [1, 30], color_continuous_scale='Viridis', width=800, height=800)

# fig.update_traces(marker=dict(size=12), selector=dict(mode='markers'))

# fig.show()

In [14]:
import tqdm

from multiprocessing import Process

In [15]:
def process_function(index):
    print("Entering process ", index)
    location_value = []
    frame = 0

    for i in range(new_output[index].shape[0]):
        for j in range(new_output[index].shape[1]):
            location_value.append([i, j, new_output[index, i, j]])

    df = pd.DataFrame(data = location_value, columns=['x', 'y', 'value'])

    # df = df[df['value']>0.1]

    df['color'] = df['value'].apply(estimate_color)

    fig = px.scatter(df, x='x', y='y', color='color', range_color = [1, 30], color_continuous_scale='Viridis', width=800, height=800)

    fig.update_traces(marker=dict(size=12), selector=dict(mode='markers'))

    fig.write_image(f'/scratch/prs392/capstone/original_simulation_2d/network_nikhil_num5_output/{index}.png', width=800, height=800)
    
    print("Exiting process ", index)
    

In [16]:
# for index in tqdm.tqdm(range(new_output.shape[0])):
#     location_value = []
#     frame = 0

#     for i in range(new_output[index].shape[0]):
#         for j in range(new_output[index].shape[1]):
#             location_value.append([i, j, new_output[index, i, j]])

#     df = pd.DataFrame(data = location_value, columns=['x', 'y', 'value'])

#     # df = df[df['value']>0.1]

#     df['color'] = df['value'].apply(estimate_color)

#     fig = px.scatter(df, x='x', y='y', color='color', range_color = [1, 30], color_continuous_scale='Viridis', width=800, height=800)

#     fig.update_traces(marker=dict(size=12), selector=dict(mode='markers'))

#     fig.write_image(f'/scratch/prs392/capstone/original_simulation_2d/network_num19_output/{index}.png', width=800, height=800)
    

In [ ]:
i = 0
while True:

    threads = []

    for index in tqdm.tqdm(range(i, new_output.shape[0])):
        threads.append(Process(target=process_function, args=(index, )))
        threads[-1].start()
        i += 1
        if i % 10 == 0:
            breakl

    for index, thread in tqdm.tqdm(enumerate(threads)):
        thread.join()
        print(f'thread {index} done')
        
    if i > new_output.shape[0]:
        break
    


  0%|          | 0/407 [00:00<?, ?it/s]

Entering process 

  0%|          | 2/407 [00:00<00:22, 18.39it/s]

 Entering process  0Entering process 

  1%|          | 4/407 [00:00<00:21, 18.67it/s]

...


 247


0it [00:00, ?it/s]

Entering process 
248
 249


Process Process-250:
Process Process-247:
Process Process-249:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-15-77f889aec6ea>", line 20, in process_function
    fig.write_image(f'/scratch/prs392/capstone/

Exiting process  247
